<a href="https://colab.research.google.com/github/chakshu-agarwal/Investments-Summary/blob/main/Robinhood_Data_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandasql

In [ ]:
pip install robin_stocks

In [4]:
import robin_stocks as r
from robin_stocks.globals import LOGGED_IN, SESSION, OUTPUT
import pyotp
import requests
import json
import pandas as pd
import pandasql as ps
from datetime import datetime
from google.colab import files

# authenticate google drive access
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())
from gspread_dataframe import set_with_dataframe

In [ ]:
# Replace MFA Key, Email and Password with the ones associated with your Robinhood account.

login = r.login('Email','Password') 

# Use the following if you have 2FA enabled on your account. Uncomment the code below
# totp  = pyotp.TOTP("[MFA Key]").now()
# login = r.login('[Email]','[Password]', mfa_code=totp)

In [8]:
"""Takes a stock instrument url and returns the ticker symbol associated with the stock.
    :param id: A string that represents the stocks instrument url.
    :type id: str
    :returns:  The symbol.
    """
symbol_for_stock = lambda url: requests.get(url).json()['symbol']

In [ ]:
# Raw dataset with all filled orders

filled_orders = []
filled_orders.append(['symbol','latest_price', 'order_type','quantity','unit_price','total_price','fees','timestamp'])
# raw = r.orders.get_all_stock_orders()
for order in raw:
  if order['executions'] != []:
    order_details = []
    order_details.append(symbol_for_stock(order['instrument']))
    try:
      price = r.stocks.get_latest_price(symbol_for_stock(order['instrument']), includeExtendedHours=True)[0]
    except:
      price = '0'
    order_details.append(price)
    order_details.append(order['side'])
    order_details.append(order['cumulative_quantity'])
    order_details.append(order['average_price'])
    order_details.append(order['executed_notional']['amount'])
    order_details.append(order['fees'])
    order_details.append(order['updated_at'])
    filled_orders.append(order_details)

# Converting to dataframe for analysis
order_data = pd.DataFrame(filled_orders2[1:], columns=filled_orders2[0])
order_data

In [ ]:
# Summary Information on all filled orders by Ticker Symbol

query = "WITH summary AS ( \
  SELECT symbol,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN quantity END) AS FLOAT) as buy_quantity,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN total_price END)/SUM(CASE WHEN order_type = 'buy' THEN quantity END) AS FLOAT) as avg_buy_price,\
  CAST(SUM(CASE WHEN order_type = 'buy' THEN total_price END) AS FLOAT) as money_out,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN quantity END) AS FLOAT) as sell_quantity,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN total_price END)/SUM(CASE WHEN order_type = 'sell' THEN quantity END) AS FLOAT) as avg_sell_price,\
  CAST(SUM(CASE WHEN order_type = 'sell' THEN total_price END) AS FLOAT) as money_in,\
  CAST(MAX(latest_price) AS FLOAT) as latest_price,\
  MAX(timestamp) as last_txn_time \
FROM order_data \
GROUP BY symbol \
) \
SELECT \
  *,\
  CASE WHEN (sell_quantity>=buy_quantity) THEN 'Closed' ELSE 'Open'END AS Position, \
  CASE WHEN (sell_quantity>=buy_quantity) THEN 0 ELSE ((buy_quantity - COALESCE(sell_quantity,0))*avg_buy_price) END AS money_still_invested, \
  CAST(money_in - (sell_quantity*avg_buy_price) AS DECIMAL) AS realized_gains_loss,\
  CASE \
    WHEN buy_quantity != sell_quantity AND sell_quantity IS NOT NULL AND latest_price IS NOT NULL \
      THEN ((buy_quantity-sell_quantity)*latest_price) - ((buy_quantity-sell_quantity)*avg_buy_price) \
    WHEN sell_quantity IS NULL AND latest_price != 'None' \
      THEN (buy_quantity*latest_price) - (buy_quantity*avg_buy_price) \
    WHEN sell_quantity IS NULL AND latest_price IS NULL \
      THEN 0 - (buy_quantity*avg_buy_price) \
    ELSE 0 \
  END AS unrealized_gains_loss \
FROM summary;"
summary_all_orders =  ps.sqldf(query)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
summary_all_orders

In [28]:
# Push Account Trading History to Google Sheets

title = '[Sheet Name]' # E.g., 'Stocks Profit/Loss'

# Assuming a spreadsheet with that title already exists
sheet = gc.open(title)
try:
  sheet = gc.open(title).add_worksheet("Robinhood_Download", rows=len(summary_all_orders)+100, cols=len(summary_all_orders.columns)+10)
  set_with_dataframe(sheet, summary_all_orders) 
except:
  sheet = gc.open(title).worksheet("Robinhood_Download")
  set_with_dataframe(sheet, summary_all_orders)

In [ ]:
# Summary Key Results
print('Money Invested: ', summary_all_orders['money_out'].sum())
print('Money Return: ', summary_all_orders['money_in'].sum())
print('Money Still Invested: ', summary_all_orders['money_still_invested'].sum())
print('Realized Gain/Loss: ', summary_all_orders['realized_gains_loss'].sum())
print('UnRealized Gain/Loss: ', summary_all_orders['unrealized_gains_loss'].sum())


In [ ]:
# YTD calculation
# current_year = "\"" + str(datetime.now().year) +"\""
current_year = '"2020"'
query = "WITH dataset AS (SELECT A.symbol,\
  A.buy_quantity,\
  A.avg_buy_price,\
  A.money_out,\
  B.sell_quantity,\
  B.avg_sell_price,\
  B.money_in,\
  B.sell_txn_year \
FROM (SELECT symbol,\
    SUM(CASE WHEN order_type = 'buy' THEN quantity END) as buy_quantity,\
    SUM(CASE WHEN order_type = 'buy' THEN total_price END)/SUM(CASE WHEN order_type = 'buy' THEN quantity END) as avg_buy_price,\
    SUM(CASE WHEN order_type = 'buy' THEN total_price END) as money_out \
  FROM order_data WHERE order_type = 'buy' GROUP BY symbol) A \
LEFT JOIN (SELECT symbol,\
    SUM(CASE WHEN order_type = 'sell' THEN quantity END) as sell_quantity,\
    SUM(CASE WHEN order_type = 'sell' THEN total_price END)/SUM(CASE WHEN order_type = 'sell' THEN quantity END) as avg_sell_price,\
    SUM(CASE WHEN order_type = 'sell' THEN total_price END) as money_in,\
    strftime('%Y',timestamp) as sell_txn_year \
  FROM order_data WHERE order_type = 'sell' AND strftime('%Y',timestamp) == {} GROUP BY symbol) B \
ON A.symbol = B.symbol) \
SELECT symbol,\
  (sell_quantity*avg_buy_price) as money_out,\
  money_in,\
  (money_in)-(sell_quantity*avg_buy_price) as realized_gains_loss \
FROM dataset \
WHERE sell_quantity IS NOT NULL \
GROUP BY symbol;".format(current_year)

ytd_data =  ps.sqldf(query)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
ytd_data

In [ ]:
# Summary Key Results
print('Realized Gain/Loss: ', ytd_data['realized_gains_loss'].sum())

In [29]:
# Push Account Realized History to Google Sheets

title = '[Sheet Name]' # E.g., 'Stocks Profit/Loss'

# Assuming a spreadsheet with that title already exists
sheet = gc.open(title)
try:
  sheet = gc.open(title).add_worksheet("Robinhood_Download_Realized_YTD", rows=len(ytd_data)+100, cols=len(ytd_data.columns)+10)
  set_with_dataframe(sheet, ytd_data) 
except:
  sheet = gc.open(title).worksheet("Robinhood_Download_Realized_YTD")
  set_with_dataframe(sheet, ytd_data)

In [ ]:
# Next Steps:

# 1. Account for stock splits and acquisitions in stock price - i can do this by including another column for current quantity held and multiply that with the latest price
# 2. Account for options
# 3. Account for repurchase of same stock at a later date - 
# 3.1 buy and sold all stake, and then buy again: sort by ticker symbol and txn date. keep doing average price if leading row is also buy, else complete the row and start a new one. For sell, do average price if only one buy row, or do average price for units up the next buy row. Update txn date to coalesce(last sales date, last buy date) for each row
# 4. Tag investments by category (long terms - growth stocks, etf, short term - gambling money) to identify future investment split
# 5. Account for fees
# 6. Add other accounts (schwab)
# 7. Add UI layer